In [ ]:
# Numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Dataset
import keras.datasets as ds

# Unconditional GAN
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, BatchNormalization, UpSampling2D

# Conditional GAN
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate

# Visualization tools
from IPython.display import clear_output, Image

In [ ]:
# Load selected dataset
def load_dataset(dataset):

    if dataset == 'MNIST':
        (trainX, trainy), (_, _) = ds.mnist.load_data()

    if dataset == 'Fashion-MNIST':
        (trainX, trainy), (_, _) = ds.fanshion_mnist.load_data()

    # Exapnd to 3D adding one channel
    X = np.expand_dims(trainX, axis=-1)

    # Convert from int to float and rescale from [0, 255] to [-1, 1]
    X = X.astype('float32')
    X = (X - (255 / 2)) / (255 / 2)
    return [X, trainy]



# Build Discriminator
def build_discriminator(input_shape):

    # Initialize the NN
    model = Sequential()

    # First convolutional layer
    model.add(Conv2D(128, 3, strides=2, padding='same', input_shape=input_shape))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))

    # Second convolutional layer
    model.add(Conv2D(256, 3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))

    # Flattening and output layer
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model



# Build Generator with default latent_space=100
def build_generator(latent_dim=100):

    # Initialize the NN
    model = Sequential()

    # Fully connected layer
    model.add(Dense(7 * 7 * 128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    model.add(Dropout(0.4))

    # First upsampling layer 14x14
    model.add(UpSampling2D())
    model.add(Conv2D(128, 3, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))

    # Second upsampling layer 28x28
    model.add(UpSampling2D())
    model.add(Conv2D(128, 3, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))

    # Output layer
    model.add(Conv2D(1, 7, activation='tanh', padding='same'))
    return model



# Build the GAN framework
def build_gan(input_shape, latent_dim):

    opt = Adam(lr=0.0002, beta_1=0.5)

    # Build Discriminator and Generator
    D = build_discriminator(input_shape=input_shape)
    G = build_generator(latent_dim=latent_dim)
    G.compile(loss='binary_crossentropy', optimizer=opt)

    # Freeze discriminator weights during generator training
    D.trainable = False

    # Connect generator and discriminator
    GAN = Sequential()
    GAN.add(G)
    GAN.add(D)
    GAN.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return GAN

In [ ]:
def gen_real(dataset, batch_size):
    X = dataset[np.random.randint(0, dataset.shape[0], size=batch_size), :, :, :]
    y = np.ones((batch_size, 1))
    return X, y


def gen_fake(G, latent_dim, batch_size):
    z = np.random.randn(latent_dim * batch_size)
    z = z.reshape(batch_size, latent_dim)
    X = G.predict(z)
    y = np.zeros((batch_size, 1))
    return X, y


# Train the GAN
def train(D, G, GAN, dataset, latent_dim=100, epochs=20, batch_size=128):
    
    batch_per_epoch = int(dataset.shape[0] / batch_size)
    half_batch = int(batch_size / 2)
    
    D_loss = []
    D_acc = []
    G_loss = []
    G_acc = []
    

    # Loop over epochs
    for epoch in range(epochs):
        
        eD_loss = []
        eD_acc = []
        eG_loss = []
        eG_acc = []
        
        for mbatch in range(batch_per_epoch):

            # Random select half_batch real samples
            realX, realY = gen_real(dataset, half_batch)
            
            # Random generate half_batch fake samples
            fakeX, fakeY = gen_fake(G, latent_dim, half_batch)

            # Stacks all samples together and train the discriminator
            X, y = np.vstack((realX, fakeX)), np.vstack((realY, fakeY))
            D_stats = D.train_on_batch(X, y)
            
            # Generate random noise and labels to train the Generator
            Z_gan = np.random.randn(latent_dim * batch_size).reshape(batch_size, latent_dim)
            y_gan = np.ones((batch_size, 1))
            G_stats = GAN.train_on_batch(Z_gan, y_gan)
            
            # Save batch parameters
            eD_loss.append(D_stats[0])
            eD_acc.append(D_stats[1])
            eG_loss.append(G_stats[0])
            eG_acc.append(G_stats[1])
        
        # Save all
        D_loss.append(eD_loss)
        D_acc.append(eD_acc)
        G_loss.append(eG_loss)
        G_acc.append(eG_acc)
        
        # Print epoch mean value
        print('Epoch-%d: dl=%.3f gl=%.3f da=%.3f ga=%.3f' %(epoch, np.mean(eD_loss), np.mean(eG_loss), np.mean(eD_acc), np.mean(eG_acc)))
        
    return D_loss, D_acc, G_loss, G_acc

In [ ]:
# Load dataset
trainX, trainy = load_dataset('MNIST')

# Define latent space
latent_space = 100
in_shape = (trainX[0].shape[0], trainX[0].shape[1], 1)

# Build Generator and Discriminator
D = build_discriminator(input_shape=in_shape)
G = build_generator(latent_dim=latent_space)

# Create DCGAN framework
GAN = build_gan(in_shape, latent_space)

D.summary()
# Train DCGAN
#D_l, D_a, G_l, G_a = train(D, G, GAN, trainX)

In [ ]:
G.summary()

In [ ]:
# Train DCGAN
D_l, D_a, G_l, G_a = train(D, G, GAN, trainX)

In [ ]:
for i in range(10):
    tmp_z = np.random.randn(latent_space).reshape(1,latent_space)

    gen = G.predict(tmp_z)
    #print(gen[0].shape)
    plt.imshow(gen[0, :, :, 0])
    plt.show()